In [1]:
# Import libraries
from collections import Counter, OrderedDict
from itertools import chain
from keras.layers import Input, Embedding, Dot, Reshape, Dense, Concatenate, Multiply
from keras.models import Model
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
import random

Using TensorFlow backend.
/Users/florencialeoni/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Path to file
hotels_path = "../data/hotels_items.csv"

In [3]:
# Create dataframe
hotels_df = pd.read_csv(hotels_path, usecols = ["item_id", "properties", "city", "price"])

In [4]:
# check info
hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727491 entries, 0 to 727490
Data columns (total 4 columns):
item_id       727491 non-null int64
properties    727491 non-null object
city          727491 non-null object
price         727491 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 22.2+ MB


In [5]:
hotels_df.head()

,item_id,properties,city,price
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Air Conditioning|Hypoallergenic Rooms|Cable TV|Hotel Bar|Pool Table|Bathtub|Satisfactory Rating|Room Service|Luxury Hotel|Terrace (Hotel)|Television|Minigolf|Business Hotel|Shower|Cot|Gym|Hairdryer|Hypoallergenic Bedding|Accessible Parking|From 3 Stars|Good Rating|Radio|4 Star|From 4 Stars|Family Friendly|Desk|Tennis Court (Indoor)|Balcony|WiFi (Public Areas)|Openable Windows|Express Check-In / Check-Out|Restaurant|Laundry Service|Ironing Board|Tennis Court|From 2 Stars|Business Centre|Bowling|Conference Rooms|Electric Kettle|Accessible Hotel|Porter|Bike Rental|Non-Smoking Rooms|Car Park|Safe (Rooms)|Fitness|Fan|Flatscreen TV|Computer with Internet|WiFi (Rooms)|Lift|Central Heating,"Mutterstadt, Germany",85.0
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Wheelchair Accessible|Hypoallergenic Rooms|Hotel Bar|Bathtub|Satisfactory Rating|Luxury Hotel|Terrace (Hotel)|Very Good Rating|Television|Business Hotel|Shower|Cot|Hairdryer|From 3 Stars|Good Rating|Radio|4 Star|From 4 Stars|Family Friendly|Desk|WiFi (Public Areas)|Openable Windows|Spa (Wellness Facility)|Laundry Service|Free WiFi (Combined)|From 2 Stars|Conference Rooms|Sauna|Bike Rental|Free WiFi (Rooms)|Non-Smoking Rooms|Car Park|Flatscreen TV|Excellent Rating|Computer with Internet|Pet Friendly|WiFi (Rooms)|Free WiFi (Public Areas)|Lift,"Passau, Germany",106.0
2,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone|Hotel|Cable TV|Hotel Bar|Bathtub|Satisfactory Rating|Room Service|Luxury Hotel|Terrace (Hotel)|Television|Business Hotel|Shower|From 3 Stars|Good Rating|Radio|4 Star|From 4 Stars|Family Friendly|Tennis Court (Indoor)|WiFi (Public Areas)|Openable Windows|Restaurant|Laundry Service|Free WiFi (Combined)|Tennis Court|From 2 Stars|Solarium|Conference Rooms|Bike Rental|Non-Smoking Rooms|Car Park|Concierge|Safe (Rooms)|Computer with Internet|Pet Friendly|Free WiFi (Public Areas)|Lift|Central Heating,"Hamburg, Germany",114.0
3,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Safe (Hotel)|Telephone|Hotel|Sitting Area (Rooms)|Reception (24/7)|Wheelchair Accessible|Hypoallergenic Rooms|Cable TV|Massage|Hotel Bar|Pool Table|Bathtub|Satisfactory Rating|Room Service|Luxury Hotel|Terrace (Hotel)|Towels|Television|Business Hotel|Shower|Steam Room|Spa Hotel|Swimming Pool (Outdoor)|Cot|Gym|Hairdryer|Beach|Hypoallergenic Bedding|Beauty Salon|Accessible Parking|From 3 Stars|Convention Hotel|Good Rating|Radio|4 Star|From 4 Stars|Table Tennis|Family Friendly|Desk|Tennis Court (Indoor)|Balcony|WiFi (Public Areas)|Surfing|Hot Stone Massage|Openable Windows|Spa (Wellness Facility)|Restaurant|Laundry Service|Ironing Board|Free WiFi (Combined)|Tennis Court|Romantic|Boat Rental|From 2 Stars|Business Centre|Solarium|Bowling|Conference Rooms|Sauna|Hammam|Accessible Hotel|Bike Rental|Free WiFi (Rooms)|Non-Smoking Rooms|Playground|Car Park|Safe (Rooms)|Fitness|Swimming Pool (Indoor)|Jacuzzi (Hotel)|Flatscreen TV|Honeymoon|Body Treat...,"Stralsund, Germany",114.0
4,6094,Satellite TV|Sailing|Safe (Hotel)|Telephone|Hotel|Hotel Bar|Bathtub|Satisfactory Rating|Luxury Hotel|Television|Minigolf|Business Hotel|Shower|Hairdryer|Good Rating|Radio|Desk|Tennis Court (Indoor)|WiFi (Public Areas)|Horse Riding|Openable Windows|Laundry Service|Tennis Court|3 Star|Non-Smoking Rooms|Car Park|Safe (Rooms)|Computer with Internet|WiFi (Rooms)|Central Heating,"Wolfsburg, Germany",108.0


In [6]:
# Create item list
item_list = hotels_df["item_id"].tolist()

# Create indexes for each item
item_index = {item: idx for idx, item in enumerate(item_list)}
index_item = {idx: item for item, idx in item_index.items()}

# Create price list
price_list = hotels_df["price"].tolist()

# Create indexes for each item
price_index = {price: idx for idx, price in enumerate(price_list)}
index_price = {idx: price for price, idx in price_index.items()}

In [7]:
# Prepare properties column

# Split of pipe
hotels_df["properties"] = hotels_df["properties"].str.split("|")

# Convert all properties to lowercase
hotels_df["properties"] = hotels_df["properties"].apply(lambda x: [w.lower() for w in x])

# Create list
properties_list = hotels_df["properties"].tolist()

# Find set of unique properties and convert to a list
unique_properties = list(chain(*[list(set(tags)) for tags in properties_list]))
unique_properties = set(unique_properties)

# Create indexes for each property
property_index = {tag: idx for idx, tag in enumerate(unique_properties)}
index_properties = {idx: tag for tag, idx in property_index.items()}

In [9]:
# Prepare city column

# Convert all properties to lowercase
hotels_df["city"] = hotels_df["city"].apply(lambda x: x.lower())

# Create list
location_list = hotels_df["city"].tolist()

# Find set of unique locations and convert to a list
# unique_locations = list(chain(*[list(set(location)) for location in location_list]))
unique_locations = set(location_list)

# Create indexes for each property
location_index = {location: idx for idx, location in enumerate(unique_locations)}
index_location = {idx: location for location, idx in location_index.items()}

In [10]:
# Build tuples to train embedding neural network
hotel_tuples = []

# Iterate through each row of dataframe
for index, row in hotels_df.iterrows():
    # Iterate through the properties in the item
    hotel_tuples.extend((item_index[hotels_df.at[index, "item_id"]], property_index[tag.lower()], 
                         price_index[hotels_df.at[index, "price"]], location_index[hotels_df.at[index, "city"]]) 
                        for tag in hotels_df.at[index, "properties"] if tag.lower() in unique_properties)

In [11]:
# Generator for training samples
def generate_batch(tuples, n_positive = 75, negative_ratio = 2.0):
    
    pairs_set = set(tuples)
    
    batch_size = n_positive * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    
    # Label for negative examples
    neg_label = 0
    
    # This creates a generator
    while True:
        # randomly choose positive examples
        for idx, (item_id, property_id, price_id, location_id) in enumerate(random.sample(tuples, n_positive)):
            batch[idx, :] = (item_id, property_id, price_id, location_id, 1)

        # Increment idx by 1
        idx += 1
        
        # Add negative examples until reach batch size
        while idx < batch_size:
            
            # random selection
            random_item = random.randrange(len(item_list))
            random_property = random.randrange(len(unique_properties))
            random_price = random.randrange(len(price_list))
            random_location = random.randrange(len(unique_locations))
            
            # Check to make sure this is not a positive example
            if (random_item, random_property, random_price, random_location) not in pairs_set:
                
                # Add to batch and increment index
                batch[idx, :] = (random_item, random_property, random_price, random_location, neg_label)
                idx += 1
                
        # Make sure to shuffle order
        np.random.shuffle(batch)
        yield {"item": batch[:, 0], "property": batch[:, 1], "price": batch[:, 2], "location": batch[:, 3]}, batch[:, 4]

In [12]:
# Properties embedding model
def hotel_embeddings(embedding_size = 100):
    
    # Inputs are one-dimensional
    item = Input(name = "item", shape = [1])
    tag = Input(name = "property", shape = [1])
    price = Input(name = "price", shape = [1])
    location = Input(name = "location", shape = [1])
    
    # Embedding the item
    item_embedding = Embedding(name = "item_embedding", input_dim = len(item_index), 
                              output_dim = embedding_size)(item)
    
    # Embedding the properties
    property_embedding = Embedding(name = "property_embedding", input_dim = len(property_index),
                                  output_dim = embedding_size)(tag)
    
    # Embedding the properties
    price_embedding = Embedding(name = "price_embedding", input_dim = len(price_index),
                                  output_dim = embedding_size)(price)
    
    # Embedding the properties
    location_embedding = Embedding(name = "location_embedding", input_dim = len(location_index),
                                  output_dim = embedding_size)(location)
    
    # Merge the embeddings with dot product across second axis
    merged_one = Dot(name = "dot_product_one", normalize = True, axes = 2)([item_embedding, property_embedding])
    merged_two = Dot(name = "dot_product_two", normalize = True, axes = 2)([item_embedding, price_embedding])
    merged_three = Dot(name = "dot_product_three", normalize = True, axes = 2)([item_embedding, location_embedding])
    merged_four = Dot(name = "dot_product_four", normalize = True, axes = 2)([price_embedding, property_embedding])
    merged_five = Dot(name = "dot_product_five", normalize = True, axes = 2)([location_embedding, property_embedding])
    merged_six = Dot(name = "dot_product_six", normalize = True, axes = 2)([location_embedding, price_embedding])
    
    # Concatenate all the merged dot products
    multiply = Multiply(name = "multiplication")([merged_one, merged_two, merged_three, merged_four, merged_five, merged_six])
    
    # Reshape to get a single number
    merged = Reshape(target_shape = [1])(multiply)
    
    merged = Dense(1, activation = "sigmoid")(merged)
    model = Model(inputs = [item, tag, price, location], outputs = merged)
    model.compile(optimizer = "Adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
    
    return model

# Instantiate model and show parameters
model = hotel_embeddings()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
property (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
price (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
location (InputLayer)           (None, 1)            0                                            
_____________________________________